# Freshwater discharge at depth in southeast Greenland fjords

## Purpose
This notebook generates figures and spreadsheets of freshwater discharge as a function of depth for each fjord in the study, and cumulatively across all fjords. 

The figures show, for each fjord, the runoff above/below a threshold depth (under `../figures/freshwater_depth_runoff_*/threshold`), and binned by depth across the whole depth range (under `../figures/freshwater_depth_runoff_*/depthbinned`), for runoff originating from either land or ice basins. There are also depth-binned figures that are cumulative across all fjords. 

The spreadsheets detail the runoff as a function of depth for each fjord (under `../databases/fjord_freshwater_flux_*`), and the mean annual runoff for each fjord. 

## Requirements
You'll need to have the following repository installed in this same folder: https://github.com/GEUS-Glaciology-and-Climate/freshwater. This should include a folder, `./freshwater`, containing `/ice` and `/land`, which each contain `/runoff` which contains MAR/RACMO data.

## Generating fjord outlets
_Note: We used v2.1 of the freshwater discharge data archived at https://doi.org/10.22008/FK2/AA6MTB. As of the publication of this repository, the data have been updated to v3 and users should use v3 if they are generating their own data. Some updates to this notebook may be required to run v3 data._

Prior to using this notebook, outlet locations were accessed using the following in the command line (I had to SSH into the APL server to make this work):

`python3 ./discharge.py --base ./freshwater --roi=./buffered_fjords/Fjord_#.kml -o > ./fjord_outlets/fjord#_outlets.txt`

In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import copy
import pathlib

In [ ]:
# USER PARAMETER: source
# Load ice and land runoff from either RACMO or MAR
source = 'MAR' # choose RACMO or MAR
runoff_ice = xr.open_mfdataset(f'./freshwater/ice/runoff/{source}_*.nc', combine='by_coords')
runoff_land = xr.open_mfdataset(f'./freshwater/land/runoff/{source}_*.nc', combine='by_coords')

In [ ]:
# Define values to be used throughout the notebook

# -- Get list of fjord numbers
fjord_number_list = list(range(1,53))

# -- Set threshold depth for above/below runoff calculations
threshold_depth = -200

# -- Define altitude intervals for depth binning
interval = 20
top = 20
shallow_bins = np.arange(threshold_depth, top+interval, interval)
all_bins = np.arange(-1000, top+interval, interval)
alt_bins = pd.interval_range(start=-1000, end=top, freq=interval)

# -- Get list of dates for timespan of data
if source == 'RACMO':
    time = pd.date_range(start='2015-01-01', end='2019-08-31')
elif source == 'MAR':
    time = pd.date_range(start='2015-01-01', end='2019-12-31')

# -- Define Blues cmap with nodata color as maroon, under_value color as silver
cmap = copy.copy(plt.cm.get_cmap('Blues'))
cmap.set_bad(color='maroon')
cmap.set_under(color='silver')

In [ ]:
def runoff_from_outlets(oi, ol):
    # MODIFIED FROM KEN MANKOFF
    ## oi & ol are outlets for ice and land respectively
    
    ## simple method, but fails if ol contains stations not in runoff_land
    ## NOTE: not sure how that occurs
    ## See https://github.com/pydata/xarray/issues/1990
    # ri = runoff_ice.sel(station = oi)
    # rl = runoff_land.sel(station = ol, method='nearest')
    ri = runoff_ice.reindex({'station':oi.id.values})
    rl = runoff_land.reindex({'station':ol.id.values})

    ri = ri.sum(dim='station').to_dataframe()['runoff']
    rl = rl.sum(dim='station').to_dataframe()['runoff']
    return ri, rl


def runoff_from_outlets_binned(oi, ol, bins, top):
    # bins can be an integer or a list (e.g. np.arange(-200, 250, 50))
    ri = runoff_ice.reindex({'station':oi.id.values})
    rl = runoff_land.reindex({'station':ol.id.values})

    if ri.alt.values.size == 0 or np.isnan(ri.alt.values).all() or (type(bins) is not int and (ri.alt.values > bins.max()).all()):
        ri_bin = None
    else: 
        ri.alt.values = np.clip(ri.alt.values, a_min=None, a_max=1) # clip all values above 0 so that they're in top bin, regardless of bin edge
        ri_bin = ri.groupby_bins(group=ri.alt, bins=bins).sum()
    if rl.alt.values.size == 0 or np.isnan(rl.alt.values).all() or (type(bins) is not int and (ri.alt.values > bins.max()).all()):
        rl_bin = None
    else: 
        rl.alt.values = np.clip(rl.alt.values, a_min=None, a_max=1) # clip all values above 0 so that they're in top bin, regardless of bin edge
        rl_bin = rl.groupby_bins(group=rl.alt, bins=bins).sum()
    return ri_bin, rl_bin

In [ ]:
# Example discharge-at-depth plot for a single fjord
fjord_number = 13
outlets = pd.read_csv(f'./fjord_outlets/fjord{fjord_number:02}_outlets.txt')
# Separate land and ice outlets
o_land = outlets.where(outlets.domain == 'land').dropna(subset=['domain'])
o_ice = outlets.where(outlets.domain == 'ice').dropna(subset=['domain'])
# Separate deep and shallow outlets (relative to a threshold depth)
o_land_deep = o_land.where(o_land.elev <= threshold_depth).dropna(subset=['elev'])
o_land_shal = o_land.where(o_land.elev > threshold_depth).dropna(subset=['elev'])
o_ice_deep = o_ice.where(o_ice.elev <= threshold_depth).dropna(subset=['elev'])
o_ice_shal = o_ice.where(o_ice.elev > threshold_depth).dropna(subset=['elev'])

oi = o_ice_shal
ol = o_land_shal

ris_bin, rls_bin = runoff_from_outlets_binned(o_ice_shal, o_land_shal, bins=shallow_bins, top=top)

fig, ax = plt.subplots()
ris_bin.runoff.plot(ax=ax, cmap=cmap, norm=colors.SymLogNorm(linthresh=1, base=10, vmin=1, vmax=ris_bin.runoff.max().values), cbar_kwargs={'label': 'Runoff ($m^3/s$)'})
start, end = ax.get_ylim()
ax.yaxis.set_ticks(np.arange(start, end, interval))
plt.show()

In [ ]:
# Generate discharge plots for each fjord

# -- Loop through each fjord and save plots of runoff above/below a threshold depth
for fjord_number in fjord_number_list:
    print(f'Processing Fjord {fjord_number}...')
    outlets = pd.read_csv(f'./fjord_outlets/fjord{fjord_number:02}_outlets.txt')

    # Separate land and ice outlets
    o_land = outlets.where(outlets.domain == 'land').dropna(subset=['domain'])
    o_ice = outlets.where(outlets.domain == 'ice').dropna(subset=['domain'])

    # Separate deep and shallow outlets (relative to a threshold depth)
    o_land_deep = o_land.where(o_land.elev <= threshold_depth).dropna(subset=['elev'])
    o_land_shal = o_land.where(o_land.elev > threshold_depth).dropna(subset=['elev'])
    o_ice_deep = o_ice.where(o_ice.elev <= threshold_depth).dropna(subset=['elev'])
    o_ice_shal = o_ice.where(o_ice.elev > threshold_depth).dropna(subset=['elev'])

    # Get runoff from shallow/deep land/ice outlets
    ris, rls = runoff_from_outlets(o_ice_shal, o_land_shal)
    rid, rld = runoff_from_outlets(o_ice_deep, o_land_deep)

    # Plot runoff above/below threshold depth for land and ice outlets
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
    axs = axs.flatten()
    fig.suptitle(f'Fjord {fjord_number}')
    # -- Plot shallow runoff
    ris.plot(ax=axs[0], label='ice')
    rls.plot(ax=axs[0], label='land')
    axs[0].legend(loc='upper left', bbox_to_anchor=(0.01, 0.99))
    axs[0].set_title(f'Shallow runoff (above {threshold_depth}m)')
    axs[0].set_ylabel('Runoff ($m^3/s$)')
    axs[0].set_xlabel('Time')
    axs[0].grid('on')
    # -- Plot deep runoff
    rid.plot(ax=axs[1], label='ice')
    rld.plot(ax=axs[1], label='land')
    axs[1].set_title(f'Deep runoff (below {threshold_depth}m)')
    axs[1].set_xlabel('Time')
    axs[1].grid('on')
    # -- Save output
    pathlib.Path(f'../figures/freshwater_depth_runoff_{source}/threshold/').mkdir(parents=True, exist_ok=True)
    plt.savefig(f'../figures/freshwater_depth_runoff_{source}/threshold/Fjord_{fjord_number:02}_threshold_runoff.png', bbox_inches='tight', dpi=300)
    plt.close()

    # Get runoff binned at depths for ice and land shallow and deep outlets
    ris_bin, rls_bin = runoff_from_outlets_binned(o_ice_shal, o_land_shal, bins=shallow_bins, top=top)
    rid_bin, rld_bin = runoff_from_outlets_binned(o_ice_deep, o_land_deep, bins=10, top=top)
    # Plot runoff time series binned by depth
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
    axs = axs.flatten()
    fig.suptitle(f'Fjord {fjord_number}')
    # -- Plot shallow ice binned runoff
    if ris_bin is not None:
        ris_bin.runoff.plot(ax=axs[0], cmap=cmap, norm=colors.SymLogNorm(linthresh=1, base=10, vmin=1, vmax=ris_bin.runoff.max().values), cbar_kwargs={'label': 'Runoff ($m^3/s$)'})
    axs[0].set_title('Shallow ice runoff')
    axs[0].set_ylabel('Altitude (m asl)')
    axs[0].xaxis.label.set_visible(False)
    start, end = axs[0].get_ylim()
    axs[0].yaxis.set_ticks(np.arange(start, end, interval))
    # -- Plot shallow land binned runoff
    if rls_bin is not None:
        rls_bin.runoff.plot(ax=axs[1], cmap=cmap, norm=colors.SymLogNorm(linthresh=1, base=10, vmin=1, vmax=rls_bin.runoff.max().values), cbar_kwargs={'label': 'Runoff ($m^3/s$)'})
    axs[1].set_title('Shallow land runoff')
    axs[1].yaxis.label.set_visible(False)
    axs[1].xaxis.label.set_visible(False)
    start, end = axs[1].get_ylim()
    axs[1].yaxis.set_ticks(np.arange(start, end, interval))
    # -- Plot deep ice binned runoff
    if rid_bin is not None:
        rid_bin.runoff.plot(ax=axs[2], cmap=cmap, norm=colors.SymLogNorm(linthresh=1, base=10, vmin=1, vmax=rid_bin.runoff.max().values), cbar_kwargs={'label': 'Runoff ($m^3/s$)'})
    axs[2].set_title('Deep ice runoff')
    axs[2].set_ylabel('Altitude (m asl)')
    axs[2].set_xlabel('Time')
    # -- Plot deep land binned runoff
    if rld_bin is not None:
        rld_bin.runoff.plot(ax=axs[3], cmap=cmap, norm=colors.SymLogNorm(linthresh=1, base=10, vmin=1, vmax=rld_bin.runoff.max().values), cbar_kwargs={'label': 'Runoff ($m^3/s$)'})
    axs[3].set_title('Deep land runoff')
    axs[3].yaxis.label.set_visible(False)
    axs[3].set_xlabel('Time')
    # -- Save output
    pathlib.Path(f'../figures/freshwater_depth_runoff_{source}/depthbinned/').mkdir(parents=True, exist_ok=True)
    plt.savefig(f'../figures/freshwater_depth_runoff_{source}/depthbinned/Fjord_{fjord_number:02}_depthbinned_runoff.png', bbox_inches='tight', dpi=300)
    plt.close()

In [ ]:
# Plot all discharge collapsed onto one plot (ice/land separated but shallow/deep not separated)

# Generate discharge plots for each fjord

# -- Initialize dataarrays for storing ice and land runoff data for all glaciers
fjordrunoff_ice = xr.DataArray(dims=['alt_bins', 'time', 'fjord'], coords=dict(alt_bins=alt_bins, time=time, fjord=fjord_number_list))
fjordrunoff_land = xr.DataArray(dims=['alt_bins', 'time', 'fjord'], coords=dict(alt_bins=alt_bins, time=time, fjord=fjord_number_list))

# -- Loop through each fjord and save plots of runoff vs depth
for fjord_number in fjord_number_list:
    print(f'Processing Fjord {fjord_number}...')
    # Generate outlets file on APL server with:
    # python3 ./discharge.py --base ./freshwater --roi=./buffered_fjords/Fjord_15.kml -o > fjord15_outlets.txt
    outlets = pd.read_csv(f'./fjord_outlets/fjord{fjord_number:02}_outlets.txt')

    # Separate land and ice outlets
    o_land = outlets.where(outlets.domain == 'land').dropna(subset=['domain'])
    o_ice = outlets.where(outlets.domain == 'ice').dropna(subset=['domain'])

    # Get runoff from shallow/deep land/ice outlets
    ri, rl = runoff_from_outlets_binned(o_ice, o_land, bins=all_bins, top=top)
    
    # Add runoff data to dataarrays
    if ri is not None:
        ri_da = xr.DataArray(ri.runoff, coords=dict(alt_bins=alt_bins, time=time, fjord=fjord_number))
        fjordrunoff_ice.loc[dict(fjord=fjord_number)] = ri_da
    if rl is not None:
        rl_da = xr.DataArray(rl.runoff, coords=dict(alt_bins=alt_bins, time=time, fjord=fjord_number))
        fjordrunoff_land.loc[dict(fjord=fjord_number)] = rl_da

# Plot runoff time series binned by depth
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(16, 7))
axs = axs.flatten()
# -- Plot binned ice runoff
fjordrunoff_ice.sum(dim='fjord').plot(ax=axs[0], cmap=cmap, norm=colors.SymLogNorm(linthresh=1, base=10, vmin=1, vmax=fjordrunoff_ice.max().values), cbar_kwargs={'label': 'Runoff ($m^3/s$)'})
axs[0].set_title('Runoff originating from ice basins')
axs[0].set_ylabel('Altitude (m asl)')
axs[0].set_xlabel('Time')
start, end = axs[0].get_ylim()
axs[0].yaxis.set_ticks(np.arange(start, end, 100))
# -- Plot binned land runoff
fjordrunoff_land.sum(dim='fjord').plot(ax=axs[1], cmap=cmap, norm=colors.SymLogNorm(linthresh=1, base=10, vmin=1, vmax=fjordrunoff_ice.max().values), cbar_kwargs={'label': 'Runoff ($m^3/s$)'})
axs[1].set_title('Runoff originating from land basins')
axs[1].set_xlabel('Time')
axs[1].yaxis.label.set_visible(False)
start, end = axs[1].get_ylim()
axs[1].yaxis.set_ticks(np.arange(start, end, 100))
# -- Save output
pathlib.Path('../figures/').mkdir(parents=True, exist_ok=True)
plt.savefig(f'../figures/runoff_{source}_depthbinned_allfjords.png', bbox_inches='tight', dpi=300)

# Plot summed ice+land runoff time series binned by depth
fjordrunoff_total = fjordrunoff_ice + fjordrunoff_land
fig, ax = plt.subplots(figsize=(8,7))
fjordrunoff_total.sum(dim='fjord').plot(ax=ax, cmap=cmap, norm=colors.SymLogNorm(linthresh=1, base=10, vmin=1, vmax=fjordrunoff_total.max().values), cbar_kwargs={'label': 'Runoff ($m^3/s$)'})
ax.set_title('Runoff')
ax.set_ylabel('Altitude (m asl)')
ax.set_xlabel('Time')
start, end = ax.get_ylim()
ax.yaxis.set_ticks(np.arange(start, end, 100))
pathlib.Path('../figures/').mkdir(parents=True, exist_ok=True)
plt.savefig(f'../figures/runoffcombined_{source}_depthbinned_allfjords.png', bbox_inches='tight', dpi=300)

In [ ]:
# Save depth-binned freshwater flux time series for each fjord to CSV
pathlib.Path(f'../databases/fjord_freshwater_flux_{source}').mkdir(parents=True, exist_ok=True)
for f in fjord_number_list:
    fjordrunoff_total.sel(fjord=f).to_pandas().transpose().to_csv(f'../databases/fjord_freshwater_flux_{source}/fjord{f:02}_freshwater_flux_binned.csv', na_rep=np.nan, header=True, index=True)

In [ ]:
# Get mean annual deep (ice+land) runoff for each fjord and save to CSV

# -- Initialize empty series to store mean annual deep runoff
deep_runoff = pd.Series(index=fjord_number_list, name='runoff', dtype='float64')

# -- Loop through each fjord and save plots of runoff vs depth
for fjord_number in fjord_number_list:
    print(f'Processing Fjord {fjord_number}...')
    # Generate outlets file on APL server with:
    # python3 ./discharge.py --base ./freshwater --roi=./buffered_fjords/Fjord_15.kml -o > fjord15_outlets.txt
    outlets = pd.read_csv(f'./fjord_outlets/fjord{fjord_number:02}_outlets.txt')

    # Separate land and ice outlets
    o_land = outlets.where(outlets.domain == 'land').dropna(subset=['domain'])
    o_ice = outlets.where(outlets.domain == 'ice').dropna(subset=['domain'])

    # Separate deep and shallow outlets (relative to a threshold depth)
    o_land_deep = o_land.where(o_land.elev <= threshold_depth).dropna(subset=['elev'])
    o_ice_deep = o_ice.where(o_ice.elev <= threshold_depth).dropna(subset=['elev'])

    # Get runoff from shallow/deep land/ice outlets
    rid, rld = runoff_from_outlets(o_ice_deep, o_land_deep)

    # Get mean annual combined (ice+land) runoff at depth
    rd = rid + rld
    rd = rd * 86400 # convert from m3/s to m3/day
    mean_annual_deep_runoff = rd.groupby(rd.index.year).sum().mean()
    deep_runoff[fjord_number] = mean_annual_deep_runoff

pathlib.Path('../databases/').mkdir(parents=True, exist_ok=True)
deep_runoff.to_csv(f'../databases/fjords_mean_annual_deep_runoff_{source}.csv', index_label='fjord', header=['mean annual runoff (m3)'])